https://kosis.kr/statHtml/statHtml.do?orgId=381&tblId=DT_920005_B008&vw_cd=MT_ZTITLE&list_id=M2_25&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_ZTITLE

In [1]:
import pandas as pd

In [2]:
df_comp = pd.read_csv("data/kosis-airline-company.csv", encoding="cp949", dtype={"시점":"string"})
df_comp.head(2)

,항공사별(1),시점,도착출발별(1),운항 (편),여객 (명),화물 (톤)
0,아시아나항공,2005.01,도착,6613,791439,28012
1,아시아나항공,2005.01,출발,6610,799835,29576


In [3]:
df_region = pd.read_csv("data/kosis-airline-region.csv", encoding="cp949", dtype={"시점":"string"})
df_region.head(2)

,지역별(1),도착출발별(1),시점,운항 (편),여객 (명),화물 (톤)
0,일본,도착,2005.01,1733,351340,16217
1,일본,도착,2005.02,1662,361438,16566


In [4]:
df_comp.columns

Index(['항공사별(1)', '시점', '도착출발별(1)', '운항 (편)', '여객 (명)', '화물 (톤)'], dtype='object')

In [5]:
df_comp.columns = ['항공사', '시점', '도착출발', '운항_편', '여객_명', '화물_톤']
df_comp.head(1)

,항공사,시점,도착출발,운항_편,여객_명,화물_톤
0,아시아나항공,2005.01,도착,6613,791439,28012


In [6]:
df_region.columns = ['지역', '도착출발', '시점', '운항_편', '여객_명', '화물_톤']
df_region.head(2)

,지역,도착출발,시점,운항_편,여객_명,화물_톤
0,일본,도착,2005.01,1733,351340,16217
1,일본,도착,2005.02,1662,361438,16566


In [7]:
df_comp["시점"].unique()

<StringArray>
['2005.01', '2005.02', '2005.03', '2005.04', '2005.05', '2005.06', '2005.07',
 '2005.08', '2005.09', '2005.10',
 ...
 '2022.11', '2022.12', '2023.01', '2023.02', '2023.03', '2023.04', '2023.05',
 '2023.06', '2023.07', '2023.08']
Length: 224, dtype: string

In [8]:
def find_year_month(df_raw):
    df_raw["연도"] = df_raw["시점"].str[:4].astype(int)
    df_raw["월"] = df_raw["시점"].str.split(".", expand=True)[1].astype(int)

In [9]:
find_year_month(df_comp)
df_comp.head(1)

,항공사,시점,도착출발,운항_편,여객_명,화물_톤,연도,월
0,아시아나항공,2005.01,도착,6613,791439,28012,2005,1


In [10]:
find_year_month(df_region)
df_region.head(1)

,지역,도착출발,시점,운항_편,여객_명,화물_톤,연도,월
0,일본,도착,2005.01,1733,351340,16217,2005,1


In [11]:
df_comp["항공사"].value_counts()

항공사
아시아나항공     448
대한항공       448
외항사        448
제주항공       414
진에어        364
에어부산       358
티웨이항공      312
이스타항공      284
에어인천       252
에어서울       172
플라이강원       86
에어로케이항공     62
에어프레미아      48
Name: count, dtype: int64

In [12]:
pd.crosstab(df_comp["항공사"], df_comp["연도"])

연도,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
항공사,,,,,,,,,,,,,,,,,,,
대한항공,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,16
아시아나항공,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,16
에어로케이항공,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,24,16
에어부산,0,0,0,6,24,24,24,24,24,24,24,24,24,24,24,24,24,24,16
에어서울,0,0,0,0,0,0,0,0,0,0,0,12,24,24,24,24,24,24,16
에어인천,0,0,0,0,0,0,0,0,20,24,24,24,24,24,24,24,24,24,16
에어프레미아,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,24,16
외항사,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,16
이스타항공,0,0,0,0,24,24,24,24,24,24,24,24,24,24,24,8,0,0,12


In [13]:
year_company = pd.crosstab(df_comp["연도"], df_comp["항공사"], values=df_comp["여객_명"], aggfunc="sum").fillna(0)
year_company.style.background_gradient(cmap="Blues", axis=None).format("{:,.0f}")

항공사,대한항공,아시아나항공,에어로케이항공,에어부산,에어서울,에어인천,에어프레미아,외항사,이스타항공,제주항공,진에어,티웨이항공,플라이강원
연도,,,,,,,,,,,,,
2005,"34,721,098","18,535,331",0,0,0,0,0,"11,319,209",0,0,0,0,0
2006,"34,990,013","19,284,957",0,0,0,0,0,"12,856,097",0,"508,612",0,0,0
2007,"34,654,664","19,352,387",0,0,0,0,0,"15,483,064",0,"1,649,508",0,0,0
2008,"33,096,441","19,786,251",0,"205,580",0,0,0,"14,579,456",0,"1,955,455","329,510",0,0
2009,"30,397,856","18,103,278",0,"2,841,444",0,0,0,"11,579,865","1,996,549","2,902,584","2,431,545",0,0
2010,"33,095,526","19,470,441",0,"4,134,712",0,0,0,"13,495,430","3,726,008","3,960,525","3,009,247","308,876",0
2011,"31,644,789","19,692,263",0,"4,632,692",0,0,0,"14,590,031","3,366,115","5,365,058","3,096,266","2,999,202",0
2012,"32,412,913","20,541,440",0,"4,955,130",0,0,0,"15,982,369","3,918,272","6,551,702","3,893,786","3,498,629",0
2013,"30,837,092","21,158,822",0,"5,855,989",0,0,0,"17,819,459","4,639,327","7,567,220","4,655,414","4,106,794",0


In [14]:
year_company = pd.crosstab(df_comp["연도"], df_comp["항공사"], values=df_comp["운항_편"], aggfunc="sum").fillna(0)
year_company.style.background_gradient(cmap="Greens", axis=None).format("{:,.0f}")

항공사,대한항공,아시아나항공,에어로케이항공,에어부산,에어서울,에어인천,에어프레미아,외항사,이스타항공,제주항공,진에어,티웨이항공,플라이강원
연도,,,,,,,,,,,,,
2005,"211,868","149,283",0,0,0,0,0,"80,798",0,0,0,0,0
2006,"212,738","146,808",0,0,0,0,0,"98,587",0,"9,568",0,0,0
2007,"220,360","145,843",0,0,0,0,0,"126,258",0,"27,152",0,0,0
2008,"217,270","156,893",0,"2,988",0,0,0,"122,093",0,"25,850","4,176",0,0
2009,"205,682","138,039",0,"29,782",0,0,0,"87,488","16,339","29,684","20,238",0,0
2010,"205,596","134,779",0,"37,230",0,0,0,"95,422","28,387","26,449","18,761","3,286",0
2011,"204,322","138,614",0,"38,162",0,0,0,"103,867","26,739","32,030","20,072","19,789",0
2012,"214,577","141,423",0,"39,710",0,0,0,"113,579","30,227","39,696","24,249","21,502",0
2013,"210,492","147,175",0,"46,911",0,416,0,"125,595","33,275","45,558","29,018","24,049",0


In [15]:
year_company = pd.crosstab(df_comp["연도"], df_comp["항공사"], values=df_comp["화물_톤"], aggfunc="sum").fillna(0)
year_company.style.background_gradient(cmap="Oranges", axis=None).format("{:,.0f}")

항공사,대한항공,아시아나항공,에어로케이항공,에어부산,에어서울,에어인천,에어프레미아,외항사,이스타항공,제주항공,진에어,티웨이항공,플라이강원
연도,,,,,,,,,,,,,
2005,"1,819,380","685,632",0,0,0,0,0,"856,586",0,0,0,0,0
2006,"1,879,078","762,478",0,0,0,0,0,"919,935",0,"2,554",0,0,0
2007,"1,954,382","788,058",0,0,0,0,0,"1,020,715",0,"7,968",0,0,0
2008,"1,814,978","781,787",0,"2,688",0,0,0,"894,448",0,"10,437","1,548",0,0
2009,"1,829,704","755,899",0,"22,502",0,0,0,"763,007","10,088","17,570","11,060",0,0
2010,"1,984,831","854,546",0,"29,819",0,0,0,"922,326","18,540","24,025","15,141","1,418",0
2011,"1,940,378","849,214",0,"35,402",0,0,0,"895,267","17,466","31,583","17,754","13,305",0
2012,"1,813,203","861,968",0,"37,500",0,0,0,"917,498","22,318","43,824","24,860","18,162",0
2013,"1,687,503","953,876",0,"43,707",0,"2,909",0,"929,790","27,465","53,750","31,009","21,623",0


In [16]:
df_region.head(1)

,지역,도착출발,시점,운항_편,여객_명,화물_톤,연도,월
0,일본,도착,2005.01,1733,351340,16217,2005,1


In [17]:
year_company = pd.crosstab(df_region["연도"], df_region["지역"], values=df_region["여객_명"], aggfunc="sum").fillna(0)
year_company.style.background_gradient(cmap="Blues", axis=None).format("{:,.0f}")

지역,기타,대양주,미주,아시아,아프리카,유럽,일본,중국,중동
연도,,,,,,,,,
2005,"20,744","921,328","3,031,358","8,459,487",0,"1,905,156","8,628,365","6,592,300","256,738"
2006,"3,479","963,031","3,144,959","9,789,110",0,"2,053,488","9,174,579","7,342,405","376,978"
2007,"12,622","1,021,044","3,368,217","10,926,890",0,"2,375,743","9,621,864","9,384,914","324,067"
2008,"6,050","1,167,561","3,300,343","10,674,781",0,"2,571,712","9,513,066","7,932,666","356,420"
2009,382,"1,130,061","3,369,417","9,390,751",0,"2,360,253","9,973,180","7,079,824","378,319"
2010,845,"1,294,206","3,700,286","11,653,773",0,"2,588,514","11,238,163","9,192,973","596,363"
2011,7,"1,492,327","3,895,005","13,785,928",212,"2,861,057","10,419,380","9,769,438","648,084"
2012,48,"1,748,255","4,014,662","15,877,264","15,537","3,190,470","11,566,437","10,832,322","715,296"
2013,0,"1,871,132","4,146,479","17,389,315","35,005","3,410,959","11,000,917","12,634,743","801,421"
